In [44]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GG_API_KEY')
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [45]:
from typing import Annotated, TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [46]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` 
    # function in the annotation defines how this state should 
    # be updated (in this case, it appends new messages to the 
    # list, rather than replacing the previous messages)
    messages: Annotated[list, add_messages]
builder = StateGraph(State)

In [47]:
def chatbot(state: State):
    answer = llm.invoke(state['messages'])
    print(answer)
    return {'messages': [answer]}

# The first argument is the unique node name
# The second argument is the function or Runnable to run
builder.add_node('chatbot', chatbot)

In [48]:
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)
graph = builder.compile()

In [49]:
img = graph.get_graph().draw_mermaid_png()
filename = "my_langgraph_workflow.png"
try:
    with open(filename, 'wb') as f:
        f.write(img)
except:
    print('Error')

In [50]:
from langchain_core.messages import HumanMessage

input = {'messages': [HumanMessage('hi!')]}
answer = graph.invoke(input)
print(answer)

content='Hi there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--653e2490-cb71-42ba-8c9a-1b52c9552207-0' usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}}
{'messages': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='2b84e31d-6432-4680-b2a9-43e53b3e05f0'), AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--653e2490-cb71-42ba-8c9a-1b52c9552207-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})]}


In [51]:
from langgraph.checkpoint.memory import MemorySaver
graph = builder.compile(checkpointer=MemorySaver())

In [52]:
thread1 = {"configurable": {"thread_id": "1"}}
result_1 = graph.invoke(
    { "messages": [HumanMessage("hi, my name is Jack!")] }, 
    thread1
)
## { "chatbot": { "messages": [AIMessage("How can I help you, Jack?")] } }

result_2 = graph.invoke(
    { "messages": [HumanMessage("what is my name?")] }, 
    thread1
)
## { "chatbot": { "messages": [AIMessage("Your name is Jack")] } }

content="Hi Jack! It's nice to meet you. How can I help you today?" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--f954ab38-13b8-4250-9ce5-5050cdb6cd8f-0' usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}
content='Your name is Jack. You just told me! 😊' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--f2d4f439-e754-4b0a-9be7-76e362d57e55-0' usage_metadata={'input_tokens': 30, 'output_tokens': 12, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}}


In [53]:
print(result_1)

{'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='30dfedc2-af4a-4a2f-b8c5-369dfc39642e'), AIMessage(content="Hi Jack! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f954ab38-13b8-4250-9ce5-5050cdb6cd8f-0', usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})]}


In [54]:
print(result_2)

{'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='30dfedc2-af4a-4a2f-b8c5-369dfc39642e'), AIMessage(content="Hi Jack! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f954ab38-13b8-4250-9ce5-5050cdb6cd8f-0', usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='1af8b8ef-34dc-4cf2-9af3-fc634b40b424'), AIMessage(content='Your name is Jack. You just told me! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f2d4f439-e754-4b0a-9be7-76e362d57e55

In [55]:
graph.get_state(thread1)

StateSnapshot(values={'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='30dfedc2-af4a-4a2f-b8c5-369dfc39642e'), AIMessage(content="Hi Jack! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f954ab38-13b8-4250-9ce5-5050cdb6cd8f-0', usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='1af8b8ef-34dc-4cf2-9af3-fc634b40b424'), AIMessage(content='Your name is Jack. You just told me! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f2d4f439-e754-4

In [56]:
graph.update_state(thread1, {'messages':[HumanMessage('I like LLMs!')]})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f054e51-3e58-6a90-8005-54a9512e02f1'}}

In [57]:
graph.get_state(thread1)

StateSnapshot(values={'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='30dfedc2-af4a-4a2f-b8c5-369dfc39642e'), AIMessage(content="Hi Jack! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f954ab38-13b8-4250-9ce5-5050cdb6cd8f-0', usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='1af8b8ef-34dc-4cf2-9af3-fc634b40b424'), AIMessage(content='Your name is Jack. You just told me! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f2d4f439-e754-4

In [58]:
answer = graph.invoke({ "messages": [HumanMessage("What do i like?")] }, 
    thread1)

content='You like LLMs!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--6240cf56-b3c4-4477-9b3d-86a47c7c366b-0' usage_metadata={'input_tokens': 51, 'output_tokens': 6, 'total_tokens': 57, 'input_token_details': {'cache_read': 0}}


In [59]:
print(answer['messages'][-1])

content='You like LLMs!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--6240cf56-b3c4-4477-9b3d-86a47c7c366b-0' usage_metadata={'input_tokens': 51, 'output_tokens': 6, 'total_tokens': 57, 'input_token_details': {'cache_read': 0}}


#### Trimming Messages

In [60]:
from langchain_core.messages import trim_messages

trimmer = trim_messages(
    max_tokens=30,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human'
)

In [61]:

from langchain_core.messages import SystemMessage, AIMessage
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

#### Filtering Messages

In [62]:
from langchain_core.messages import filter_messages
messages = [
    SystemMessage("you are a good assistant", id="1"),
    HumanMessage("example input", id="2", name="example_user"),
    AIMessage("example output", id="3", name="example_assistant"),
    HumanMessage("real input", id="4", name="bob"),
    AIMessage("real output", id="5", name="alice"),
]

filter_messages(messages, include_types="human")

[HumanMessage(content='example input', additional_kwargs={}, response_metadata={}, name='example_user', id='2'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4')]

In [63]:
filter_messages(messages, exclude_names=["example_user", "example_assistant"])

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}, id='1'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4'),
 AIMessage(content='real output', additional_kwargs={}, response_metadata={}, name='alice', id='5')]

In [64]:
filter_messages(
    messages, 
    include_types=[HumanMessage, AIMessage], 
    exclude_ids=["3"]
)

[HumanMessage(content='example input', additional_kwargs={}, response_metadata={}, name='example_user', id='2'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4'),
 AIMessage(content='real output', additional_kwargs={}, response_metadata={}, name='alice', id='5')]

In [65]:
filter_ = filter_messages(exclude_names=["example_user", "example_assistant"])

chain = filter_ | llm 

chain.invoke(messages)

AIMessage(content='', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--85c8c0b4-7d51-498d-aaa9-6b566fb67496-0', usage_metadata={'input_tokens': 9, 'output_tokens': 0, 'total_tokens': 9, 'input_token_details': {'cache_read': 0}})

#### Merging Consecutive Messages

In [66]:
from langchain_core.messages import merge_message_runs

messages = [
    SystemMessage("you're a good assistant."),
    SystemMessage("you always respond with a joke."),
    HumanMessage(
        [{"type": "text", "text": "i wonder why it's called langchain"}]
    ),
    HumanMessage("and who is harrison chasing anyway"),
    AIMessage(
        '''Well, I guess they thought "WordRope" and "SentenceString" just 
        didn\'t have the same ring to it!'''
    ),
    AIMessage("""Why, he's probably chasing after the last cup of coffee in the 
        office!"""),
]

merge_message_runs(messages)

[SystemMessage(content="you're a good assistant.\nyou always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content=[{'type': 'text', 'text': "i wonder why it's called langchain"}, 'and who is harrison chasing anyway'], additional_kwargs={}, response_metadata={}),
 AIMessage(content='Well, I guess they thought "WordRope" and "SentenceString" just \n        didn\'t have the same ring to it!\nWhy, he\'s probably chasing after the last cup of coffee in the \n        office!', additional_kwargs={}, response_metadata={})]